KDE-based empirical prediction of melt/nomelt with tas
===

Global module imports
---

In [1]:
# %matplotlib inline
# %config InlineBackend.print_figure_kwargs = {'bbox_inches':'tight'}
# from plotUtils import PlotUtils 

from netCDF4 import Dataset,date2num
import pandas as pd
import numpy as np
from scipy import stats
import os
# import seaborn as sns
# import matplotlib.pyplot as plt
# import matplotlib.ticker as ticker
# from matplotlib.lines import Line2D
# from matplotlib.text import Text

%load_ext autoreload
%autoreload 2
from model import Model
from GCNet import GCNet
from itertools import izip

pd.options.mode.chained_assignment = None

Function: define upper/lower temperature limits
---

In [2]:
def setLimits(  mms, var, src = "grid" ):
    '''
        bounds based on KDE of various datasets, 1986-2015
        note: this is a doubly-nested dictionary!
    '''
    bounds = { \
                'tas': { \
                   'aws': { \
                       '06':  ( -7.8, -0.1 ), \
                       '07':  ( -7.2, 0.3 ), \
                       '08':  ( -8.8, 0.3 ), \
                       'JJA': ( -7.7, 0.2 ) \
                          }, \
                   'grid': { \
                       '06':  ( -7.7, -0.7 ), \
                       '07':  ( -6.9, -0.4 ), \
                       '08':  ( -8.7, -0.4 ), \
                       'JJA': ( -7.6, -0.4 ) \
                           } \
                       }, \
                'tasmax': { \
                  'aws': { \
                      '06':  ( -2.9, 1.2 ), \
                      '07':  ( -2.1, 1.4 ), \
                      '08':  ( -3.8, 1.4 ), \
                      'JJA': ( -2.8, 1.3 ) \
                         }, \
                  'grid': { \
                      '06':  ( -3.7, 0.9 ), \
                      '07':  ( -2.5, 1.0 ), \
                      '08':  ( -4.5, 1.9 ), \
                      'JJA': ( -3.3, 1.0 ) \
                          } \
                      } \
                 }
    try:
        return bounds[var][src][mms]
    except:
        print "setLimits: Key error(s)!  Args: var =",var,", src =",src,", mms =",mms
        return (None, None)

Function: Predict melt from temperature
---

In [3]:
def predictMelt( M, lwr, upr ):
    """ predict melt from temperature using empirical transition function"""

    # setup
    melt = np.zeros_like( M ).astype(np.float32)
    
    # set points with obvious melt 
    flg = M >= upr
    melt[flg] = 1

    # interpolate ambiguous melt points
    flg = (M >= lwr) & (M < upr)
    x = (M[flg] - lwr) / (upr - lwr)
    melt[flg] = x
    
    return melt

Global data
---

In [4]:
# key data
var = "tas"
# var = "tasmax"

model = "erai"
branch = "historical"
src = "grid"
calibYears = "1986-2015"

# mon = 6
# mon = 7
# mon = 8
mon = "JJA"

# ------------------------------------
# derived data
if branch == "historical":
    yr1 = "1996"
    yr2 = "2005"

if branch == "rcp85":
    yr1 = "2071"
    yr2 = "2080"

try:
    mms = "%02d" % mon
except:
    mms = mon

yrs = yr1+"-"+yr2

File templates etc
---

In [5]:
modelMetaFN = "wrf_geog.nc"

modelDataDir = "/Volumes/sbp1/model/pwrf/gis_%s/%s/wrf/postproc/%s" % ( model, branch, var )
modelDataFN = "%s_wrf_%s_%s_%s_d.nc" % ( var, model, yrs, mms )

newFileTemplate = "wrf_%s_tas_%%s_%s_%s.nc" % ( model, yrs, mms )
newFile = newFileTemplate % ( "beta" )

Load temperature data
----

In [6]:
print "Temperature data:", modelDataFN
D = Model(None, modelDataDir+"/"+modelDataFN)

time, timeCF = D.loadData( "time" )
print "Time range:", time[0], time[-1]
# dfTime = pd.Series( time, name="Time")

X = np.array( D.loadData( var ) )
nRec, nLat, nLon = X.shape
print "Shape:", nRec, nLat, nLon

Temperature data: tas_wrf_erai_1996-2005_JJA_d.nc
Time range: 1996-06-01 10:30:00 2005-08-31 10:30:00
Shape: 920 219 197


Load Mote ice sheet mask
---

In [7]:
fn = "icesheet_mask.nc"
ncfn = Dataset(fn, "r")
icemask2d = np.array( ncfn.variables[ "icemask" ] )
icemask3d = np.repeat( icemask2d[None,:,:],nRec,axis=0)

Predict melt, then mask to ice sheet
---

In [8]:
lwr, upr = setLimits( mms, var, src )
print "Lower", lwr, "Upper", upr

meltPredF = predictMelt( X, lwr, upr )

noice = (icemask3d < 1)
meltPredF[noice] = np.nan

nRec2, nLat2, nLon2 = meltPredF.shape
print "Shape:", nRec2, nLat2, nLon2

Lower -7.6 Upper -0.4
Shape: 920 219 197


Convert from 0-1 to 0/1, i.e., turn floats into integers
---

In [9]:
# set critical value for melt/nomelt and _FillValue for integer version
pCrit = 0.5        # default, not expected to be used
if var == "tas":
    pCrit = 0.738
if var == "tasmax":
    pCrit = 0.839
intFillValue = -999
print "Critical value =",pCrit,", _FillValue =", intFillValue

# adjust based on critical value
meltNewRnd = np.where( meltPredF < pCrit, 0., 1. )

# convert to integer and retain missing values
nanix = np.isnan(meltPredF)
meltPredI = np.rint( meltNewRnd ).astype(np.int16)
meltPredI[nanix] = intFillValue  # can't use np.nan since that's only for floats!

Critical value = 0.738 , _FillValue = -999


/Library/Python/2.7/site-packages/ipykernel/__main__.py:11: RuntimeWarning: invalid value encountered in less


Calculate a "total melt days" variable
---

In [11]:
meltSum = np.sum( meltPredI, axis = 0 )
meltSum = np.where( meltSum < 0, 0, meltSum )

Create a netCDF file for the new data
-----

In [13]:
# new file will have TWO melt variables:
# 1. meltProb = melt prediction as floating point probability
# 2. melt     = melt prediction as 0/1 integer based on critical probability cutoff value
print "Output file:", newFile

try: 
    ncfile.close()  # just to be safe, make sure dataset is not already open.
except: 
    pass

ncfile = Dataset(newFile, mode='w', format = 'NETCDF4_CLASSIC')

# global metadata
ncfile.title = "Predicted melt occurrence"
ncfile.bounds = "%.2f, %.2f" % (lwr, upr)
ncfile.source_model = model
ncfile.source_data = src
ncfile.branch = branch
ncfile.period = yrs
ncfile.month = mms
ncfile.calibYears = calibYears
ncfile.predictorVar = var
bw = 0.4
ncfile.kdeBandWidth = bw

time_dim = ncfile.createDimension('time', None)
sn_dim  = ncfile.createDimension('south_north', nLat)
we_dim  = ncfile.createDimension('west_east', nLon)

timeVar = ncfile.createVariable('time', np.float32, ('time',))
timeVar.units = timeCF.units
timeVar.calendar = timeCF.calendar
timeVar.standard_name = timeCF.standard_name
timeVar.long_name = timeCF.long_name
timeVar.axis = timeCF.axis
timeVar[:] = date2num(time, timeCF.units, timeCF.calendar)

# melt as floating point 0-1
meltVarProb = ncfile.createVariable('meltProb', np.float32, ('time','south_north','west_east'), \
                                fill_value = np.nan )
meltVarProb.description = "Melt occurrence (probability)"
meltVarProb.long_name = "Melt occurrence probability (0-1) predicted from near-surface temperature"
meltVarProb.units = "probability"
meltVarProb[:] = meltPredF

# melt as integer 0/1
meltVarOcc = ncfile.createVariable('melt', np.int16, ('time','south_north','west_east'), \
                                fill_value = intFillValue )
meltVarOcc.description = "Melt occurrence (0/1)"
meltVarOcc.long_name = "Melt occurrence (0/1) predicted from near-surface temperature"
meltVarOcc.units = "count"
meltVarOcc.critValue = pCrit
meltVarOcc[:] = meltPredI

# melt sum
meltVarSum = ncfile.createVariable('meltSum', np.int16, \
                                   ('south_north','west_east'), \
                                   fill_value = intFillValue )
meltVarSum.description = "Number of melt days"
meltVarSum.long_name = "Total number of melt days in the study period"
meltVarSum.units = "count"
meltVarSum[:] = meltSum

ncfile.close()

Output file: wrf_erai_tas_beta_1996-2005_JJA.nc
